# Problema 4.2

Problema da dieta

## Enunciado

In [1]:
# Custo por kg (R$/kg) de cada tipo de alimento
custo = {
    "Pão": 90,
    "Carne": 350,
    "Batatas": 50,
    "Legumes": 60,
    "Leite": 50,
}

# Quantidade requerida por dia de cada tipo de nutriente
demanda_nutrientes = {
    "Calorias": 3000,  # Unidade
    "Proteínas": 70,  # g
    "Cálcio": 800,  # mg
    "Vitamina A": 500,  # UI
}

# Quantidade de cada nutriente por kg de cada tipo de alimento
nutrientes = {
    "Pão": {
        "Calorias": 2750,
        "Proteínas": 85,
        "Cálcio": 920,
        "Vitamina A": 0,
    },
    "Carne": {
        "Calorias": 3250,
        "Proteínas": 165,
        "Cálcio": 90,
        "Vitamina A": 0,
    },
    "Batatas": {
        "Calorias": 705,
        "Proteínas": 18,
        "Cálcio": 95,
        "Vitamina A": 160,
    },
    "Legumes": {
        "Calorias": 100,
        "Proteínas": 9,
        "Cálcio": 310,
        "Vitamina A": 1900,
    },
    "Leite": {
        "Calorias": 690,
        "Proteínas": 35,
        "Cálcio": 1180,
        "Vitamina A": 1600,
    },
}

## Resolução (`ortools`)

In [2]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver("GLOP")
if not solver:
    print("Solver nao encontrado")

### Variáveis de decisão:
- $x_{1}$: quantidade de `pão` comprada
- $x_{2}$: quantidade de `carne` comprada
- $x_{3}$: quantidade de `batatas` comprada
- $x_{4}$: quantidade de `legumes` comprada
- $x_{5}$: quantidade de `leite` comprada


In [3]:
x1 = solver.NumVar(0, solver.infinity(), "x1")
x2 = solver.NumVar(0, solver.infinity(), "x2")
x3 = solver.NumVar(0, solver.infinity(), "x3")
x4 = solver.NumVar(0, solver.infinity(), "x4")
x5 = solver.NumVar(0, solver.infinity(), "x5")

### Função objetivo:

Queremos minimizar o custo total da compra

In [4]:
solver.Minimize(
    custo["Pão"] * solver.LookupVariable("x1")
    + custo["Carne"] * solver.LookupVariable("x2")
    + custo["Batatas"] * solver.LookupVariable("x3")
    + custo["Legumes"] * solver.LookupVariable("x4")
    + custo["Leite"] * solver.LookupVariable("x5")
)

### Restrições:

In [5]:
# Quantidade mínima de cada nutriente deve ser respeitada

solver.Add(
    nutrientes["Pão"]["Calorias"] * solver.LookupVariable("x1")
    + nutrientes["Carne"]["Calorias"] * solver.LookupVariable("x2")
    + nutrientes["Batatas"]["Calorias"] * solver.LookupVariable("x3")
    + nutrientes["Legumes"]["Calorias"] * solver.LookupVariable("x4")
    + nutrientes["Leite"]["Calorias"] * solver.LookupVariable("x5")
    >= demanda_nutrientes["Calorias"]
)

solver.Add(
    nutrientes["Pão"]["Proteínas"] * solver.LookupVariable("x1")
    + nutrientes["Carne"]["Proteínas"] * solver.LookupVariable("x2")
    + nutrientes["Batatas"]["Proteínas"] * solver.LookupVariable("x3")
    + nutrientes["Legumes"]["Proteínas"] * solver.LookupVariable("x4")
    + nutrientes["Leite"]["Proteínas"] * solver.LookupVariable("x5")
    >= demanda_nutrientes["Proteínas"]
)

solver.Add(
    nutrientes["Pão"]["Cálcio"] * solver.LookupVariable("x1")
    + nutrientes["Carne"]["Cálcio"] * solver.LookupVariable("x2")
    + nutrientes["Batatas"]["Cálcio"] * solver.LookupVariable("x3")
    + nutrientes["Legumes"]["Cálcio"] * solver.LookupVariable("x4")
    + nutrientes["Leite"]["Cálcio"] * solver.LookupVariable("x5")
    >= demanda_nutrientes["Cálcio"]
)

solver.Add(
    nutrientes["Pão"]["Vitamina A"] * solver.LookupVariable("x1")
    + nutrientes["Carne"]["Vitamina A"] * solver.LookupVariable("x2")
    + nutrientes["Batatas"]["Vitamina A"] * solver.LookupVariable("x3")
    + nutrientes["Legumes"]["Vitamina A"] * solver.LookupVariable("x4")
    + nutrientes["Leite"]["Vitamina A"] * solver.LookupVariable("x5")
    >= demanda_nutrientes["Vitamina A"]
)

# Não negatividade das variáveis de decisão
solver.Add(solver.LookupVariable("x1") >= 0)
solver.Add(solver.LookupVariable("x2") >= 0)
solver.Add(solver.LookupVariable("x3") >= 0)
solver.Add(solver.LookupVariable("x4") >= 0)
solver.Add(solver.LookupVariable("x5") >= 0)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x00000172B7733D20> >

### Otimização do sistema

In [6]:
print("Number of variables =", solver.NumVariables())
print("Number of constraints =", solver.NumConstraints())

Number of variables = 5
Number of constraints = 9


In [7]:
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print("Solution:")
    print(f"Objective value = {solver.Objective().Value():0.1f}")
    print(f"x1 = {x1.solution_value():0.2f}")
    print(f"x2 = {x2.solution_value():0.2f}")
    print(f"x3 = {x3.solution_value():0.2f}")
    print(f"x4 = {x4.solution_value():0.2f}")
    print(f"x5 = {x5.solution_value():0.2f}")
else:
    print("The problem does not have an optimal solution.")

Solving with Glop solver v9.10.4067
Solution:
Objective value = 106.8
x1 = 1.01
x2 = 0.00
x3 = 0.00
x4 = 0.00
x5 = 0.31


### Resolução (`CPLEX`)

In [8]:
import docplex.mp.model as cpx

model = cpx.Model(name="alimentos")

# Variáveis de decisão
x1 = model.continuous_var(name="x1")
x2 = model.continuous_var(name="x2")
x3 = model.continuous_var(name="x3")
x4 = model.continuous_var(name="x4")
x5 = model.continuous_var(name="x5")

# Função objetivo
model.minimize(
    custo["Pão"] * x1
    + custo["Carne"] * x2
    + custo["Batatas"] * x3
    + custo["Legumes"] * x4
    + custo["Leite"] * x5
)

# Restrições
model.add_constraint(
    nutrientes["Pão"]["Calorias"] * x1
    + nutrientes["Carne"]["Calorias"] * x2
    + nutrientes["Batatas"]["Calorias"] * x3
    + nutrientes["Legumes"]["Calorias"] * x4
    + nutrientes["Leite"]["Calorias"] * x5
    >= demanda_nutrientes["Calorias"]
)

model.add_constraint(
    nutrientes["Pão"]["Proteínas"] * x1
    + nutrientes["Carne"]["Proteínas"] * x2
    + nutrientes["Batatas"]["Proteínas"] * x3
    + nutrientes["Legumes"]["Proteínas"] * x4
    + nutrientes["Leite"]["Proteínas"] * x5
    >= demanda_nutrientes["Proteínas"]
)

model.add_constraint(
    nutrientes["Pão"]["Cálcio"] * x1
    + nutrientes["Carne"]["Cálcio"] * x2
    + nutrientes["Batatas"]["Cálcio"] * x3
    + nutrientes["Legumes"]["Cálcio"] * x4
    + nutrientes["Leite"]["Cálcio"] * x5
    >= demanda_nutrientes["Cálcio"]
)

model.add_constraint(
    nutrientes["Pão"]["Vitamina A"] * x1
    + nutrientes["Carne"]["Vitamina A"] * x2
    + nutrientes["Batatas"]["Vitamina A"] * x3
    + nutrientes["Legumes"]["Vitamina A"] * x4
    + nutrientes["Leite"]["Vitamina A"] * x5
    >= demanda_nutrientes["Vitamina A"]
)

# Resolvendo o modelo
solution = model.solve()

print(f"Solution: {model.solve()}")
print(f"Objective value = {solution.get_objective_value():0.1f}")
print(f"x1 = {solution.get_value(x1):0.2f}")
print(f"x2 = {solution.get_value(x2):0.2f}")
print(f"x3 = {solution.get_value(x3):0.2f}")
print(f"x4 = {solution.get_value(x4):0.2f}")
print(f"x5 = {solution.get_value(x5):0.2f}")

Solution: solution for: alimentos
objective: 106.75
status: OPTIMAL_SOLUTION(2)
x1=1.012
x5=0.312

Objective value = 106.7
x1 = 1.01
x2 = 0.00
x3 = 0.00
x4 = 0.00
x5 = 0.31


O CPLEX e o Google ORTools encontraram o mesmo valor de solução ótima para o problema.

In [9]:
model.export_as_lp("problema_4.2.lp")

'problema_4.2.lp'